# Multi-objective Bayesian Optimisation

https://botorch.org/docs/multi_objective
* "The goal in MOBO is learn the Pareto front: the set of optimal trade-offs, where an improvement in one objective means deteriorating another objective."
* "The MC-based acquisition functions support using the sample average approximation for rapid convergence"
* Additionally, variations on ParEGO can be trivially implemented using an augmented Chebyshev scalarization as the objective with an EI-type single-objective acquisition function such as qLogNoisyExpectedImprovement. The get_chebyshev_scalarization convenience function generates these scalarizations.
  
https://botorch.org/tutorials/constrained_multi_objective_bo
Considerations
* BoTorch assumes maximization
  * leave growth rate as it is
  * maximise negative of costs


from: https://botorch.org/tutorials/constrained_multi_objective_bo

"For batch optimization (or in noisy settings), we strongly recommend using NEHVI rather than EHVI [1] because it is far more efficient than EHVI and mathematically equivalent in the noiseless setting." <- but that's hypervolumen...

"Note: EHVI aggressively exploits parallel hardware and is much faster when run on a GPU. See [1] for details."
-> use GPU and run on server

### Function
* The model consists of a list of mono-objective models.
The likelihood at each point is the sum of all (both) GP’s likelihood.
* Acquisition optimisation
  * optimises multiple (2) objectives at once
  * approach used: qNParEGO
    * starts with scalarisation - combine all objectives into a single compound function (augmented chebyshev)
  * random process; possibly slower than qEHVI or qNEHVI
  *
### Concept
Within the function
1. Change the medium composition using BayesOpt
2. For each composition
     1. calculate the cost
     2. find optimal growth rate using FBA
     3. calculate growth/cost (should be maximised)
        * How to prevent that the cost is driven to 0? -> even if it drives growth to 0, it will probably be numerically optimal
3. Return optimal medium composition, growth rate, costs and relationship

# Implementation

In [2]:
# When running in google colab
#pip install cobra

In [3]:
# When running in google colab
#pip install botorch

In [ ]:
import torch
from botorch.fit import fit_gpytorch_mll
# sampler
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.sampling import sample_simplex

# Acquisition function
from botorch.optim.optimize import optimize_acqf_list # for qPAREGO
from botorch.optim.optimize import optimize_acqf # for qEHVI and qNEHVI

from botorch.acquisition.logei import qLogExpectedImprovement # qPAREGO, assumes noiselessness
# qEHVI, assumes noiselessness
from botorch.acquisition.multi_objective.monte_carlo import qExpectedHypervolumeImprovement 
from botorch.acquisition.multi_objective.logei import qLogExpectedHypervolumeImprovement
# more efficient for batches than qEHVI, math. equivalent in noiseless setting
from botorch.acquisition.multi_objective.monte_carlo import qNoisyExpectedHypervolumeImprovement 
from botorch.acquisition.multi_objective.logei import qLogNoisyExpectedHypervolumeImprovement
from botorch.acquisition.objective import GenericMCObjective

from botorch.utils.multi_objective.box_decompositions.non_dominated import FastNondominatedPartitioning

from botorch.utils.transforms import unnormalize, normalize # for normalising media components
from botorch.utils.multi_objective.scalarization import get_chebyshev_scalarization
from botorch.utils.multi_objective.pareto import is_non_dominated # pareto

import random # for initial data
import json # to save results
import copy # to be able to do deep copies
import numpy as np

### Helper Functions & Plotting

In [ ]:
# Helper functions to be used across notebooks
%run HelperFunctions_MOBO.ipynb

# same but for .py version
#from HelperFunctions_MOBO import *

## BayesOpt

In [ ]:
# called with normalised growth, cost and production
def find_next_candidates(
        medium_tensors_normalised_stacked, 
        growth_tensors,
        bounds_tensors_stacked, 
        n_candidates = 5,
        opt_objective = "growth-cost",
        cost_tensors = None, 
        production_tensors = None,
        AF_type = 'qPAREGO'
        ):
    """
    Finds the next medium composition for which to evaluate cost and optimal growth rate
    * normalises the medium composition between 0,1
    * initialises botorch model (list of SingleTaskGP) and mll
    * calculates posterior mean
    * uses SobolQMCNormalSampler to sample from posterior
    * gives choice of methods
    * qPAREGO
        * uses qLogExpectedImprovement acquisition function
        * uses chebyshev_scalarization to create a vector representation of the two objectives (cost mini, growth rate max)
    * qEHVI
        * uses qLogExpectedHypervolumeImprovement
    * qNEHVI
        * uses qNoisyExpectedHypervolumeImprovement
    
    PARAMETERS
    * medium_tensors_normalised_stacked - tensor - all medium compositions previously evaluated, stored as tensors (in order)
    * growth_tensors - tensor - corresponding growth rates
    * bounds_tensors_stacked - tensors - upper and lower bounds for the values the medium components are allowed to take,
    determines the search space;
    * n_candidates - integer - how many candidates to find at once = batch_size
    * opt_objective - string - the multi-objective
    * cost_tensors - tensor - corresponding medium costs
    * production_tensors - tensor - corresponding production rates
    * AF_type - string - which acquisition function to use

    RETURNS
    * candidates[0] - tensor - a tensors with the normalised (0,1) medium composition
    """
    # assert that the objective is one of the possibilities
    opt_objective_types = ['growth-cost', 'growth-production', 'growth-production-cost',]
    if opt_objective not in opt_objective_types:
        raise ValueError("Invalid objective. Expected one of: %o" % opt_objective_types)
    
    '''parameters and conversion to tensors; normalisation of medium composition to (0,1)'''
    MC_SAMPLES = 256 # Number of Monte Carlo samples in SobolQMCNormalSampler
    n_components = medium_tensors_normalised_stacked.size()[1] # of medium components
    # bounds to normalise medium components (x) to
    standard_bounds = standard_bounds = torch.tensor([[0.0] * n_components, 
                                                      [1.0] * n_components]).to(**tkwargs) # normalised bounds for medium composition
    # large values -> slower but possibly better accuracy
    NUM_RESTARTS =  10 #10 # Number of restarts for acquisition function optimisation
    RAW_SAMPLES = 1024 # 1024 # Number of raw samples for initialisation of acquisition optimisation


    '''Setup'''
    # Set up a Sobol quasi-Monte Carlo sampler for sampling from the posterior
    # The sample_shape should correspond to the shape of the posterior samples needed
    sampler = SobolQMCNormalSampler(sample_shape = torch.Size([MC_SAMPLES]), seed = MC_SAMPLES)
    
    # initialise GP model and marginal likelihood (mll)

    mll, model = initialise_model(
        medium_tensors_normalised_stacked, 
        growth_tensors, 
        opt_objective = opt_objective,
        cost_tensors = cost_tensors, 
        production_tensors = production_tensors)

    # throws min-max error or normalisation errors, when x or y not formatted correctly
    fit_gpytorch_mll(mll) # Fit the model using the maximum marginal likelihood
    
    print("INSIDE find_next_candidate", 
          "\nGrowth:", growth_tensors,
          "\nCosts:", cost_tensors,
          "\nProduction", production_tensors, 
          sep ="\n")
    
    '''finding the new candidate'''
    if AF_type == 'qPAREGO':
        # Compute the posterior mean for the given medium_tensors_stacked using the model
        with torch.no_grad():
            posterior = model.posterior(medium_tensors_normalised_stacked).mean
        
        acq_fun_list = [] # List to hold acquisition functions for each candidate

        if opt_objective == "growth-cost":
            # Loop to generate acquisition functions for each candidate
            for _ in range(n_candidates):
                # Sample weights from the simplex for Chebyshev scalarization
                weights = sample_simplex(2, **tkwargs).squeeze() # using 2 weights for scalarization (cost and growth)

                # Compute the scalarised objective values for all the training points
                scalarized_objective_values = (
                    weights[0] * growth_tensors + 
                    weights[1] * cost_tensors)
                # Find the best observed scalarized objective value
                best_f = scalarized_objective_values.max().item()

                # Define objectivenfor AF
                objective = GenericMCObjective(
                    get_chebyshev_scalarization(weights, posterior)
                )
                print("Best F:", best_f,
                      "\nWeights:", weights,
                      "\nPosterior:", posterior,
                      "\nObjective:", objective,
                      sep = "\n")
                # Define the acquisition function using quasi Monte Carlo EI
                acq_fun = qLogExpectedImprovement(
                    model = model, # List of SingleTastk GP
                    best_f = best_f, # best objective value observed so far - replaces X_baseline in Noisy version
                    sampler = sampler, # SobolQMCNormalSampler
                    objective = objective, # combination of cost and growth - Chebyshev scalarization
                )
                acq_fun_list.append(acq_fun)

        elif opt_objective == "growth-production":
            # Loop to generate acquisition functions for each candidate
            for _ in range(n_candidates):
                # Sample weights from the simplex for Chebyshev scalarization
                weights = sample_simplex(2, **tkwargs).squeeze() # using 2 weights for scalarization (growth and production)

                # Compute the scalarised objective values for all the training points
                scalarized_objective_values = (weights[0] * growth_tensors + weights[1] * production_tensors)
                # Find the best observed scalarized objective value
                best_f = scalarized_objective_values.max().item()

                # Define objective for AF
                objective = GenericMCObjective(
                    get_chebyshev_scalarization(weights, posterior)
                )

                # Define the acquisition function using quasi Monte Carlo EI
                acq_fun = qLogExpectedImprovement(
                    model = model, # List of SingleTastk GP
                    best_f = best_f, # best objective value observed so far - replaces X_baseline in Noisy version
                    sampler = sampler, # SobolQMCNormalSampler
                    objective = objective, # combination of growth and production - Chebyshev scalarization
                )
                acq_fun_list.append(acq_fun)
        
        elif opt_objective == "growth-production-cost":
            # Loop to generate acquisition functions for each candidate
            for _ in range(n_candidates):
                # Sample weights from the simplex for Chebyshev scalarization
                weights = sample_simplex(3, **tkwargs).squeeze() # using 3 weights for scalarization (cost, production and growth)

                # Compute the scalarised objective values for all the training points
                scalarized_objective_values = (
                    weights[0] * growth_tensors + 
                    weights[1] * production_tensors +
                    weights[2] * cost_tensors)
                # Find the best observed scalarized objective value
                best_f = scalarized_objective_values.max().item()

                # Define objective for AF
                objective = GenericMCObjective(
                    get_chebyshev_scalarization(weights, posterior)
                )

                # Define the acquisition function using quasi Monte Carlo EI
                acq_fun = qLogExpectedImprovement(
                    model = model, # List of SingleTastk GP
                    best_f = best_f, # best objective value observed so far - replaces X_baseline in Noisy version
                    sampler = sampler, # SobolQMCNormalSampler
                    objective = objective, # combination of cost and growth - Chebyshev scalarization
                )
                acq_fun_list.append(acq_fun)

        candidates, _ = optimize_acqf_list(
            acq_function_list = acq_fun_list,  # List of acquisition functions to optimise
            bounds = standard_bounds, # The normalised bounds for optimisation
            num_restarts = NUM_RESTARTS, # Number of restarts for optimisation
            raw_samples = RAW_SAMPLES, # Number of raw samples for initialisation (?)
            options = {"batch_limit": 10, "maxiter": 200,} # Options for acquisition function optimisation
        )

        return candidates
    
    elif ((AF_type == 'qEHVI') or (AF_type == 'qNEHVI')):
        """ Reference Point
        EHVI requires specifying a reference point, which is the lower bound on the objectives 
        used for computing hypervolume. 
        In practice the reference point can be set 
        1) using domain knowledge to be slightly worse than the lower bound of objective values, 
        where the lower bound is the minimum acceptable value of interest for each objective, 
        or 
        2) using a dynamic reference point selection strategy.
        """
        # TODO: adjust ref_point
        # working with normalised y's -  maybe shouldn't? 
        # or fix ref point and scale it with normalisation bound
        if opt_objective == "growth-cost":
            ref_point = torch.tensor([0.05, -0.95]) # just to see if it runs
            # non-normalised
            #ref_point = torch.tensor([0.5, -2000])
        
        elif opt_objective == "growth-production":
            ref_point = torch.tensor([0.05, 0.05]) # just to see if it runs
            # non-normalised
            #ref_point = torch.tensor([0.2, 0.001])

        elif opt_objective == "growth-production-cost":
            ref_point = torch.tensor([0.05, 0.05, -0.95]) # just to see if it run
            # non-normalised
            #ref_point = torch.tensor([0.2, 0.001, -3000])


        if AF_type == 'qEHVI':
            '''def optimize_qehvi_and_get_observation(model, train_x, train_obj, sampler):'''
            # Compute the posterior mean for the given medium_tensors_stacked using the model
            with torch.no_grad():
                posterior = model.posterior(medium_tensors_normalised_stacked).mean

            '''FastNondominatedPartitioning'''
            # assumes maximization
            # Parameters:
            # ref_point (Tensor) – A m-dim tensor containing the reference point.
            # Y (Optional[Tensor]) – A (batch_shape) x n x m-dim tensor.        
            partitioning = FastNondominatedPartitioning(
                ref_point = ref_point, Y = posterior
                )

            # Define the acquisition function using qEHVI
            """
            acq_func = qExpectedHypervolumeImprovement(
                model = model,
                ref_point = ref_point,
                partitioning = partitioning,
                sampler = sampler,
                )
            """
            # Try qLogEHVI, as recommended
            acq_func = qLogExpectedHypervolumeImprovement(
                model = model,
                ref_point = ref_point,
                partitioning = partitioning,
                sampler = sampler,
                )
        
        # q-Log Noisy Expected Hypervolume Improvement supporting m>=2 outcomes.
        elif AF_type == 'qNEHVI':
            # partition non-dominated space into disjoint rectangles
            """
            acq_func = qNoisyExpectedHypervolumeImprovement(
                model = model,
                ref_point = ref_point.tolist(),  # use known reference point
                X_baseline = medium_tensors_normalised_stacked, # normalize(train_x, problem.bounds)
                prune_baseline = True,  # prune baseline points that have estimated zero probability of being Pareto optimal
                sampler = sampler,
            )
            """
            # Try qLogEHVI, as recommended
            acq_func = qLogNoisyExpectedHypervolumeImprovement(
                model = model,
                ref_point = ref_point.tolist(),  # use known reference point
                X_baseline = medium_tensors_normalised_stacked, # normalize(train_x, problem.bounds)
                prune_baseline = True,  # prune baseline points that have estimated zero probability of being Pareto optimal
                sampler = sampler,
             )
            
        # optimize
        candidates, _ = optimize_acqf(
            acq_function = acq_func, # qEHVI
            bounds = standard_bounds, # The normalised bounds for optimisation
            q = n_candidates,
            num_restarts = NUM_RESTARTS, # Number of restarts for optimisation
            raw_samples = RAW_SAMPLES, # used for intialization heuristic
            options = {"batch_limit": 20, "maxiter": 200}, # Options for acquisition function optimisation
            sequential = True,
            )
            
        return candidates

### Main

In [ ]:
def media_BayesOpt(
        MetModel, medium = None, bounds = None, costs = None,
        opt_objective = "growth-cost",
        biomass_objective = None, 
        production_objective = None,
        n_start = 20, n_iter = 50, n_candidates = 1,
        AF_type = 'qPAREGO',
        ):
    """
    Performs medium optimisation for various objectives
    * trade-off between growth rate and cost
    * trade-off between growth rate and production

    1. Sets default values for medium, bounds and costs if not provided by the user
    2. Performs optimisation n_iter (default = 50) times
        1. calls generate_initial_data(args) to generate initial data points
        2. finds new candidate medium calling find_next_candidate(args)
        3. evaluates new medium for growth rate and costs
        4. keeps all values
    3. returns optimal composition alongside corresponding cost, growth, and cost-growth trade-off

    PARAMETERS:
    * MetModel - COBRApy model - the metabolic model to be evaluated
    * medium - dictionary - the medium composition of that model; if not provided defaults to default medium provided by CobraPy
    * bounds - dictionary - upper and lower bounds for the values the medium components are allowed to take,
    determines the search space; if not provided defaults to 0, and current medium value
    * costs - dictionary - the (monetary) cost of each component; if not provided defaults to unit costs
    * opt_objective - string - indicates what is to be optimised
    * biomass_objective - string - id of the biomass vector
    * production_objective - string - id of the production reaction to be optmised
    * n_start - integer - how many random media compositions are to be created to set up the BayesOpt
    * n_iter - integer - how many candidate medium compositions should be found and evaluated
    * n_candidates - integer - batch size; how many candidates to find per iteration
    - AF_type - string - which acquisitin function to use

    RETURNS:
    A dictionary containing
    * the acquisition function used
    * the objective used (or combination of objectives)
    * a dictionary with the upper and lower bounds of each medium components
    * a dictionary with the cost of each medium component
    * a list of all evaluated medium compositions
    * a tensor with corresponding total medium costs
    * a tensor with corresponding growth rates
    * a binary tensor indicating if a  medium composition is on the pareto front
    """

    # assert that the objective is one of the possibilities
    opt_objective_types = ['growth-cost', 'growth-production', 'growth-production-cost',]
    if opt_objective not in opt_objective_types:
        raise ValueError("Invalid objective. Expected one of: %o" % opt_objective_types)
    
    # assert that the acquisition function type is one of the possibilities
    AF_types = ['qPAREGO', 'qEHVI', 'qNEHVI']
    if AF_type not in AF_types:
        raise ValueError("Invalid acquisition function. Expected one of: %o" % AF_types)

    # Set default values for medium, boundaries and costs
    if medium is None:
        medium = MetModel.medium  # Default medium to model.medium if not provided
    if bounds is None:
        # if no bounds are provided, set the lower limit to 0 and upper to value in medium
        #bounds = {key: (0, 2*medium[key]+100) for key in medium.keys()} # used up until 09.10.2024
        bounds = {key: (0, medium[key]) for key in medium.keys()}
    if costs is None:
        costs = {key: 1 for key in medium.keys()}  # Default unit cost if not provided
    
    # TODO: check that model.medium and costs have the same number of entries

    # objective of metabolic model should always be to maximise growth
    if biomass_objective is None:
        raise ValueError("Please specifiy the biomass objective.")
    MetModel.objective_direction = 'max' 
    MetModel.objective = biomass_objective  

    '''initialisation samples'''
    # generate n_start initial data points (parameters and corresponding cost + growth rate)
    initial_para, initial_growth, initial_production, initial_cost = generate_initial_data(
        MetModel, medium, bounds, costs, n_samples = n_start,
        opt_objective = objective,
        biomass_objective = biomass_objective, 
        production_objective = production_objective
        )
   
    # medium
    medium_list = initial_para # list of dictonaries
    medium_keys = medium_list[-1].keys() # extract keys from medium_list
    # stack bounds; convert medium list to tensors and normalise it (0,1)
    bounds_tensors_stacked, medium_tensors_normalised = convert_normalise_media(bounds, medium_list)
    
    '''reassign and normalise'''
    # growth
    growth_tensors = initial_growth # tensor
    growth_tensors_normalised = normalise_1Dtensors(growth_tensors)
    # cost
    cost_tensors = initial_cost # tensor | <0 (generate_initial_data returns negative total costs)
    # multiply by -1 to get pos. values, normalise, multiply with -1 again to get neg. values
    cost_tensors_normalised = (normalise_1Dtensors(cost_tensors*(-1))*(-1))
    # production
    production_tensors = initial_production
    production_tensors_normalised = torch.tensor([])
    if production_objective != None:
        production_tensors_normalised = normalise_1Dtensors(production_tensors)

    is_pareto = torch.tensor([]) # empty

    print("\nINITIAL:",
          "\nGrowth:", growth_tensors, growth_tensors_normalised,
          "\nCosts:", cost_tensors, cost_tensors_normalised,
          "\nProduction", production_tensors, production_tensors_normalised,
          sep ="\n")



    # Optimise the trade-off between growth and cost
    if opt_objective == "growth-cost":

        # MAIN LOOP
        for i in range(n_iter):
            # Stack the list of tensors along a new dimension (dim=0) -> single tensor
            medium_tensors_normalised_stacked = torch.stack(medium_tensors_normalised, dim = 0) # normalised
            
            # Use BayesOpt to change medium
            # normalised growth and cost
            candidates_tensor_normalised = find_next_candidates(
                medium_tensors_normalised_stacked, 
                growth_tensors_normalised,
                bounds_tensors_stacked,
                n_candidates = n_candidates,
                opt_objective = opt_objective,
                cost_tensors = cost_tensors_normalised,
                production_tensors = None,
                AF_type = AF_type
                )
            
            """
            # without normalisation of output (growth, cost)
            candidates_tensor_normalised = find_next_candidates(
                medium_tensors_normalised_stacked, 
                growth_tensors,
                bounds_tensors_stacked,
                n_candidates = n_candidates,
                opt_objective = opt_objective,
                cost_tensors = cost_tensors,
                production_tensors = None,
                AF_type = AF_type
                )
            """
            # evaluate each candidate medium using slim.optimize() - faster than optimize()
            for j in range(n_candidates):
                candidate_tensor_normalised = candidates_tensor_normalised[j]
                # unnormlise new candidate
                candidate_tensor_unnormalised = unnormalize(candidate_tensor_normalised, bounds_tensors_stacked)
                # convert back to dictionary            
                candidate_medium = convert_to_dict(candidate_tensor_unnormalised, medium_keys)
                
                
                # for new medium compute new values
                cost_tot = calc_cost_tot(costs, candidate_medium) # tensor

                MetModel.medium = candidate_medium # assign new medium to model
                growth = MetModel.slim_optimize() # calculate growth rate - float
                # some model compositions lead to MetModel.slim_optimze returning NaN
                # to avoid them from breaking the algorithm, set growth to zero
                # same goes for negative growth which breaks normalisation
                if (np.isnan(growth) or (growth < 0.0)):
                    growth = 0

                '''append to list/tensor and normalise resulting vector (min/max might have changed -> compute from scratch)'''
                # medium lists
                medium_list.append(candidate_medium)
                medium_tensors_normalised.append(candidate_tensor_normalised)
                # cost
                cost_tot = -cost_tot  # BOtorch assumes maximisation, so we maximise the negative of the costs.
                cost_tensors = torch.cat((cost_tensors, cost_tot), dim=0)  # Concatenate along dimension 0 (1D tensors)
                cost_tensors_normalised = (normalise_1Dtensors(cost_tensors*(-1))*(-1))
                # growth
                growth_tensor = torch.tensor([growth], dtype=torch.double).to(**tkwargs)
                growth_tensors = torch.cat((growth_tensors, growth_tensor), dim=0)  # Concatenate along dimension 0
                growth_tensors_normalised = normalise_1Dtensors(growth_tensors)

            print("Iteration:\t", i+1,
                  "\nGrowth:", growth_tensors, growth_tensors_normalised,
                  "\nCosts:", cost_tensors, cost_tensors_normalised,
                  "\nProduction", production_tensors, production_tensors_normalised,
                  sep ="\n")
            
            if ((i+1)%10 == 0):
                print("Iteration:\t", i+1)


        # Find all points on pareto front and return them     
        # Stack the two objectives (growth rate and medium cost) into a single 2D tensor
        # rows: candidates
        # columns: growth rate (positive), medium costs (negative)
        y = torch.stack((growth_tensors, cost_tensors), dim = 1)        
        is_pareto = is_non_dominated(y.to(**tkwargs), maximize = True) # Compute non-dominated (Pareto front) points; i.e. optimal trade.offs
    

    # Optimise the trade-off between growth and production
    elif opt_objective == "growth-production":  
        if production_objective is None:
            raise ValueError("Please specifiy the production objective.")         

        # MAIN LOOP
        for i in range(n_iter):
            # Stack the list of tensors along a new dimension (dim=0) -> single tensor
            medium_tensors_normalised_stacked = torch.stack(medium_tensors_normalised, dim = 0) # normalised

            # Use BayesOpt to change medium
            # with  normalised output
            candidates_tensor_normalised = find_next_candidates(
                medium_tensors_normalised_stacked, 
                growth_tensors_normalised,
                bounds_tensors_stacked,
                n_candidates = n_candidates,
                opt_objective = opt_objective,
                cost_tensors = None, 
                production_tensors = production_tensors_normalised,
                AF_type = AF_type
                )
            """
            # without normalisation of output (growth, production)
            candidates_tensor_normalised = find_next_candidates(
                medium_tensors_normalised_stacked, 
                growth_tensors,
                bounds_tensors_stacked,
                n_candidates = n_candidates,
                opt_objective = opt_objective,
                cost_tensors = None, 
                production_tensors = production_tensors,
                AF_type = AF_type
                )
            """
            # evaluate each candidate medium
            for j in range(n_candidates):
                candidate_tensor_normalised = candidates_tensor_normalised[j]
                # unnormlise new candidate
                candidate_tensor_unnormalised = unnormalize(candidate_tensor_normalised, bounds_tensors_stacked)
                # convert back to dictionary            
                candidate_medium = convert_to_dict(candidate_tensor_unnormalised, medium_keys)
                
                # calculate the cost
                cost_tot = calc_cost_tot(costs, candidate_medium) # tensor
                
                # for new medium compute new values
                MetModel.medium = candidate_medium # assign candidate medium to model
                
                '''FBA'''
                # assign biomass function as objective
                MetModel.objective = biomass_objective
                FBA_solution = MetModel.optimize()
                growth = FBA_solution.fluxes[biomass_objective]
                production = FBA_solution.fluxes[production_objective]
                
                # if either is NaN or smaller than zero, set to zero
                # negative values cause problems for the scalarisation and are biologically implausible
                if (np.isnan(growth) or (growth < 0.0)):
                    growth = 0
                if (np.isnan(production) or (production < 0.0)):
                    production = 0  


                '''append to new result to list/tensor'''
                # medium lists
                medium_list.append(candidate_medium)
                medium_tensors_normalised.append(candidate_tensor_normalised)
                # growth
                growth_tensor = torch.tensor([growth], dtype=torch.double).to(**tkwargs)
                growth_tensors = torch.cat((growth_tensors, growth_tensor), dim=0)  # Concatenate along dimension 0
                growth_tensors_normalised = normalise_1Dtensors(growth_tensors)
                # production
                production_tensor = torch.tensor([production], dtype=torch.double).to(**tkwargs)
                production_tensors = torch.cat((production_tensors, production_tensor), dim=0)  # Concatenate along dimension 0
                production_tensors_normalised = normalise_1Dtensors(production_tensors)
                # cost
                cost_tot = -cost_tot  # BoTorch assumes maximisation, so we maximise the negative of the costs.
                cost_tensors = torch.cat((cost_tensors, cost_tot), dim=0)  # Concatenate along dimension 0 (1D tensors)
                cost_tensors_normalised = (normalise_1Dtensors(cost_tensors*(-1))*(-1))

            if ((i+1)%10 == 0):
                print("Iteration:\t", i+1)


        # Find all points on pareto front and return them     
        # Stack the two objectives (growth rate and production rate) into a single 2D tensor
        # rows: candidates
        # columns: growth rate (positive), production rate (positive)
        y = torch.stack((growth_tensors, production_tensors), dim = 1)
        is_pareto = is_non_dominated(y.to(**tkwargs), maximize = True) # Compute non-dominated (Pareto front) points; i.e. optimal trade.offs


    # Optimise the trade-off between growth, production and cost
    elif opt_objective == "growth-production-cost":
        if production_objective is None:
            raise ValueError("Please specifiy the production objective.") 
        
        # MAIN LOOP
        for i in range(n_iter):
            # Stack the list of tensors along a new dimension (dim=0) -> single tensor
            medium_tensors_normalised_stacked = torch.stack(medium_tensors_normalised, dim = 0) # normalised


            # Use BayesOpt to change medium
            """
            candidates_tensor_normalised = find_next_candidates(
                medium_tensors_normalised_stacked, 
                growth_tensors_normalised,
                bounds_tensors_stacked,
                n_candidates = n_candidates,
                opt_objective = opt_objective,
                cost_tensors = cost_tensors_normalised, 
                production_tensors = production_tensors_normalised,
                AF_type = AF_type
                )
            """
            # without normalisation of output (growth, production, cost)
            candidates_tensor_normalised = find_next_candidates(
                medium_tensors_normalised_stacked, 
                growth_tensors,
                bounds_tensors_stacked,
                n_candidates = n_candidates,
                opt_objective = opt_objective,
                cost_tensors = cost_tensors, 
                production_tensors = production_tensors,
                AF_type = AF_type
                )
            
            # evaluate each candidate medium
            for j in range(n_candidates):
                candidate_tensor_normalised = candidates_tensor_normalised[j]
                # unnormlise new candidate
                candidate_tensor_unnormalised = unnormalize(candidate_tensor_normalised, bounds_tensors_stacked)
                # convert back to dictionary            
                candidate_medium = convert_to_dict(candidate_tensor_unnormalised, medium_keys)
                
                # calculate the cost
                cost_tot = calc_cost_tot(costs, candidate_medium) # tensor
                
                # for new medium compute new values
                MetModel.medium = candidate_medium # assign candidate medium to model
    
                
                '''FBA'''
                # assign biomass function as objective
                MetModel.objective = biomass_objective
                FBA_solution = MetModel.optimize()
                growth = FBA_solution.fluxes[biomass_objective]
                production = FBA_solution.fluxes[production_objective]
                
                # if either is NaN or smaller than zero, set to zero
                # negative values cause problems for the scalarisation and are biologically implausible
                if (np.isnan(growth) or (growth < 0.0)):
                    growth = 0
                if (np.isnan(production) or (production < 0.0)):
                    production = 0 


                '''append to new result to list/tensor'''
                # medium lists
                medium_list.append(candidate_medium)
                medium_tensors_normalised.append(candidate_tensor_normalised)
                # growth
                growth_tensor = torch.tensor([growth], dtype=torch.double).to(**tkwargs)
                growth_tensors = torch.cat((growth_tensors, growth_tensor), dim=0)  # Concatenate along dimension 0
                growth_tensors_normalised = normalise_1Dtensors(growth_tensors)
                # production
                production_tensor = torch.tensor([production], dtype=torch.double).to(**tkwargs)
                production_tensors = torch.cat((production_tensors, production_tensor), dim=0)  # Concatenate along dimension 0
                production_tensors_normalised = normalise_1Dtensors(production_tensors)
                # cost
                cost_tot = -cost_tot  # BoTorch assumes maximisation, so we maximise the negative of the costs.
                cost_tensors = torch.cat((cost_tensors, cost_tot), dim=0)  # Concatenate along dimension 0 (1D tensors)
                cost_tensors_normalised = (normalise_1Dtensors(cost_tensors*(-1))*(-1))

            if ((i+1)%10 == 0):
                print("Iteration:\t", i+1)


        # Find all points on pareto front and return them     
        # Stack the three objectives (growth rate, production rate and cost) into a single 2D tensor
        # rows: candidates
        # columns: growth rate (positive), production rate (positive), cost (negative)
        y = torch.stack((growth_tensors, production_tensors, cost_tensors), dim = 1)   
        is_pareto = is_non_dominated(y.to(**tkwargs), maximize = True) # Compute non-dominated (Pareto front) points; i.e. optimal trade.offs
    


     # multiply cost_tensor with -1 so that it returns positive values
    
    
    return {
        "acquisition function" : AF_type,
        "objective" : opt_objective,
        "medium component bounds" : bounds, 
        "medium component costs" : costs,
        "medium list" : medium_list, 
        "cost tensors" : (cost_tensors*(-1)),
        "growth rate tensors" : growth_tensors,
        "production tensors" : production_tensors,
        "is pareto" : is_pareto,
        "biomass_objective" : biomass_objective,
        "production_objective" : production_objective
        }